In [ ]:
import os
import requests
import json
import time
import re

import pandas as pd
import polyline
import matplotlib.pyplot as plt

In [ ]:
TOKENS_FILEPATH = os.getenv("TOKENS_FILEPATH")
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

In [ ]:
def get_tokens(client_id, client_secret, tokens_filepath):
    """ Gets the Strava tokens from a json file and refreshes them if expired """

    # Get the tokens from file to connect to Strava
    with open(tokens_filepath) as json_file:
        strava_tokens = json.load(json_file)

    # If access_token has expired then use the refresh_token to get the new access_token
    if strava_tokens['expires_at'] < time.time():

        # Make Strava auth API call with current refresh token
        response = requests.post(
            url='https://www.strava.com/oauth/token',
            data={
                'client_id': int(client_id),
                'client_secret': client_secret,
                'grant_type': 'refresh_token',
                'refresh_token': strava_tokens['refresh_token']
            }
        )

        # Save response as json in new variable
        new_strava_tokens = response.json()

        # Save new tokens to file
        with open(tokens_filepath, 'w') as outfile:
            json.dump(new_strava_tokens, outfile)

        # Use new Strava tokens from now
        strava_tokens = new_strava_tokens

    return strava_tokens

In [ ]:
strava_tokens = get_tokens(CLIENT_ID, CLIENT_SECRET, TOKENS_FILEPATH)

In [ ]:
def plot_polyline(summary_polyline):
    """ Plots a segment or a route from a polyline object """

    coordinates = polyline.decode(summary_polyline)

    ride_longitudes = [coordinate[1] for coordinate in coordinates]
    ride_latitudes = [coordinate[0] for coordinate in coordinates]

    plt.plot(ride_longitudes, ride_latitudes, 'r-', alpha=1)
    plt.show()

In [ ]:
# Loop through all activities (first page)
url = "https://www.strava.com/api/v3/activities"
access_token = strava_tokens['access_token']
# Get first page of activities from Strava with all fields
r = requests.get(url + '?access_token=' + access_token)
r = r.json()
    
df = pd.json_normalize(r)

In [ ]:
# Get Activity info
#athlete_id = strava_tokens['athlete']['id']
#url = "https://www.strava.com/api/v3/athletes/{}/routes".format(athlete_id)
url = "https://www.strava.com/api/v3/activities/4074378152"
access_token = strava_tokens['access_token']

# Get first page of activities from Strava with all fields
r = requests.get(url + '?access_token=' + access_token)
r = r.json()

summary_polyline_route = r['map']['summary_polyline']
plot_polyline(summary_polyline_route)

df_segments = pd.json_normalize(r['segment_efforts'])

In [ ]:
r

In [ ]:
r['message']

In [ ]:
if "segment_efforts" in r:
    print("Loading segments from activity")
    
elif "message" in r:
    print(r['message'])

else:
    print("Unknown error")

In [ ]:
df_segments

In [ ]:
df_segments.loc[0]

In [ ]:
# get how many segments in activity
df_segments.shape

In [ ]:
def calculate_time_difference_from_leader(segment_id, athlete_elapsed_time,
                                          gender, strava_tokens):
    """
    Gets the time of the segment's leader in seconds and calculates 
    the percent difference from the anthlete time
    """
    def get_sec(time_str):
        """ Get Seconds from time """

        if time_str.find("s") == -1:
            try:
                h, m, s = time_str.split(':')
                return int(h) * 3600 + int(m) * 60 + int(s)
            except ValueError:
                m, s = time_str.split(':')
                return int(m) * 60 + int(s)
        else:
            return [int(s) for s in re.findall(r'-?\d+\.?\d*', time_str)][0]

    url = "https://www.strava.com/api/v3/segments/{}".format(segment_id)
    access_token = strava_tokens['access_token']

    # Get first page of activities from Strava with all fields
    r = requests.get(url + '?access_token=' + access_token)
    r = r.json()

    # get leader time
    leader_elapsed_time = get_sec(
        r['xoms']['qom']) if gender == 'women' else get_sec(r['xoms']['kom'])

    return athlete_elapsed_time / leader_elapsed_time - 1

In [ ]:
%%time
df_segments["segment_time_delta"] = df_segments.apply(
    lambda x: calculate_time_difference_from_leader(x["segment.id"],
                                                    x["elapsed_time"],
                                                    gender="man",
                                                    strava_tokens=strava_tokens
                                                    ),
    axis=1)

In [ ]:
df_segments.sort_values(by=['segment_time_delta'], inplace=True)

In [ ]:
df_segments

In [ ]:
activity_id = '4074378152'
segment_id = df_segments['id'][0]
url_segment = 'https://www.strava.com/activities/{}/segments/2741743541458943054'.format(
    activity_id, segment_id)

In [ ]:
import webbrowser

webbrowser.open(url_segment)

Ideas:
 * plot map with segments coloured by proximity to KOM (interactive map?)
 * create app where to select activity to get the segments analysis. Be able to plot the segment or open it in the browser
 * create REST API Flask
 * ref for plotting segment_polylines https://knanne.github.io/notebooks/visualize_strava_data_in_python.html#Geographic-Data

In [ ]:
#!pipenv install git+https://github.com/matplotlib/basemap.git

In [ ]:
# previous step:
# $ brew install geos
# !pipenv install https://github.com/matplotlib/basemap/archive/master.zip

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

map = Basemap(llcrnrlon=3.75,llcrnrlat=39.75,urcrnrlon=4.35,urcrnrlat=40.15, epsg=5520)
#http://server.arcgisonline.com/arcgis/rest/services

map.arcgisimage(service='ESRI_Imagery_World_2D', xpixels = 1500, verbose= True)
plt.show()

In [ ]:
# opens a map in a new window
import requests
from PIL import Image
url ='http://server.arcgisonline.com/ArcGIS/rest/services/ESRI_Imagery_World_2D/MapServer/export?bbox=1564270.9620172689,4401598.726055895,1615017.0560379555,4446612.184939481&bboxSR=5520&imageSR=5520&size=1500,1330&dpi=96&format=png32&transparent=true&f=image'
im = Image.open(requests.get(url, stream=True).raw)
im.show()

In [ ]:
import polyline
import matplotlib.pyplot as plt

from mpl_toolkits.basemap import Basemap

#summary_polyline = "ij_}Hyjd^dB_Gp@aAn@[F[{BwQpA?p@v@rCQf@m@_@iTSaAe\\cEWVKt@yANs@jCkAfB_JhIdEpP|ApD|BfDl@`DcEvCwEbCcHtKiGnLmKnOcClF{HxLoBrBgG~DmAnBy@pCYxBi@xIHlBu@xH{@tPu@fRLpIS~Eo@lC_HtOuJ|LwOrH{FrH}BdEoChBeBt@{HpBwBpBqA|Ba@K{DeGkKnQc@WkEwGsG~Aj@rBjBjWOhA}AtEc@`Ge\\rZyCbH{@|@Ij@r@hA|BxURf@E`@qJzQy_@fz@uGtG{Cy@}GxAyCrCqAfC{AfHmC|SQtEdAxHpBvGv@`EDbFm@pDo@r@sAl@qGc@}ALgJzIkGhNcDzOkBvHsCpD{EvCaGfBqWtC_FpB_F`FeOdV_HtEsJL{JeCqFaC}D}DkH{FyRiD_B}@cC`@{CyAeC`@}RrK_ErAcDnBuFJmBk@sA{@kAoBgD}Iw@_DK}DnAcBUsCTcIIsDkAkGu@eAiCiGyOoSoE{DqImDqE|@aG{AaH}CwDaD_DmFkCgMgB_GeEyFkV_LgE_AaI{BuARoDrBy@z@_BfDSBqCkLoBwFmBsBmDoAgCIkCxB}CtAsI?iFq@{B^cAYsAoBIcDcBkO}A{BuKcd@{Sq`AmAkH_@aFKuN^gFxAyHtPyj@fBcE|E}OzWk}@~CiHXgE`BoIv@wGj]_jE`AgBtAd@j_@j^bRdNn@|Ab@lFp@rAvJ|GxBr@^l@\\ANm@M_FJgu@l@mI`AsHfEeU`BsEjFcKxKoM|FwE~CcB`Cl@|DfH|CjGrInS`EzLnEjSrNb~@hHjc@jBnRzFv~@fCjf@jAn\\N|Ge@r}@L|Fj@t@tAXxPhBdeAkG`Du@M{DmHog@u@gDDiA`^cSdWmP|[e\\fDaE|HaGxB{D|FeO?e@_@u@sZue@YcBxOyl@nFiVtGqVnTir@r@gBd@e@VuCxCkJIgEfGyiAdAwZe@w@kQ{LYy@zDyWvDoRd@_A~zAfq@|q@l_@l@FnBaBfBe@fVvL`AxAt@tB^kEnBKlBf@~AhAr@`Av@zBtEeBfAyElAg@hGnA`B|@jBnBp@`AXbAAzDr@fCC`D`@lG\\NXfC`C`Fl@rBtGnc@Et@cA`Bn@`CyApDxBzD~@lCvCvBtEj]xBpKGp@aFnJ?b@~GpXdBdE~BnD|AxHxC~Dr@ItEdAzB_@"
summary_polyline = '_fq~Hy`d]f@qa@DkEHyClAyz@PoKRcBl@mBzCsI'

coordinates = polyline.decode(summary_polyline)

ride_longitudes = [coordinate[1] for coordinate in coordinates]
ride_latitudes = [coordinate[0] for coordinate in coordinates]

m = Basemap(
    llcrnrlon=min(ride_longitudes) - 0.02,
    llcrnrlat=min(ride_latitudes) - 0.02,
    urcrnrlon=max(ride_longitudes) + 0.02, 
    urcrnrlat=max(ride_latitudes) + 0.02,
    epsg=23095,
)

m.arcgisimage(service="World_Imagery", verbose=True)
x, y = m(ride_longitudes, ride_latitudes)
m.plot(x, y, 'r-')

plt.show()